# Lesson 4: Auto-merging Retrieval

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import utils

import os
import openai

openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 7bede8b2-a51c-4ae9-b507-baa275975e03
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Auto-merging retrieval setup

In [5]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[2048, 512, 128])

In [37]:
# Returns all levels of nodes.  This should be Leaf, Intermediate and Root (Parent) nodes.  They all form the hierachy.  The text of parent nodes is composed of the text of multiple intermeidate nodes.

nodes = node_parser.get_nodes_from_documents([document])

In [8]:
len(nodes)

144

In [36]:
from llama_index.node_parser import get_leaf_nodes, get_root_nodes

leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)
print(f"count of leaf_nodes: {len(leaf_nodes)}")
print(f"count of root_nodes: {len(root_nodes)}")
print(f"ROOT_NODE Length: {len(root_nodes[0].text)}")
print(f"LEAF_NODE Length:: {len(leaf_nodes[0].text)}")
print("ROOT_NODE: " + root_nodes[0].text)
print("LEAF_NODE: " + leaf_nodes[0].text)

count of leaf_nodes: 112
count of root_nodes: 6
ROOT_NODE Length: 9642
LEAF_NODE Length:: 444
ROOT_NODE: PAGE 1Founder, DeepLearning.AICollected Insights
from Andrew Ng
How to 
Build
Your
Career
in AIA Simple Guide


PAGE 2"AI is the new 
electricity. It will 
transform and improve 
all areas of human life."
Andrew Ng

PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Career Growth.
Chapter 2: Learning Technical Skills for a 
Promising AI Career.
Chapter 3: Should You Learn Math to Get a Job 
in AI?
Chapter 4: Scoping Successful AI Projects.
Chapter 5: Finding Projects that Complement 
Your Career Goals.
Chapter 6: Building a Portfolio of Projects that 
Shows Skill Progression.
Chapter 7: A Simple Framework for Starting Your AI 
Job Search.
Chapter 8: Using Informational Interviews to Find 
the Right Job.
Chapter 9: Finding the Right AI Job for You.
Chapter 10: Keys to Building a Career in AI.
Chapter 11: Overcoming Imposter Syndrome.
Final 

In [29]:
nodes_by_id["55c658d7-11c6-4b01-b812-14b3ce7a1bde"]

TextNode(id_='55c658d7-11c6-4b01-b812-14b3ce7a1bde', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='45c4ff9b-17b2-45d1-ab76-dc11fd8bb99f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='55bc135e5090691b219b2ac617e8e31549c7f346bac85fe72557c3c5fa640d90'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d97b48a0-7a71-434b-a09b-0717f83148ef', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b2044ddc300d7051774032da15a8f50599527cfb3809e02c1d820338593a9852'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='fbbfbf39-e24c-4499-8197-28612ed59637', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='668a0c56d7367f073644c6f438e3942ea824ca28d2e22119473a562d18b20f45'), <NodeRelationship.PARENT: '4'>: RelatedNodeInfo(node_id='45c4ff9b-17b2-45d1-ab76-dc11fd8bb99f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='55bc135e5090691b219b2ac617e8e31549c7f34

In [23]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[27].parent_node.node_id]
print(parent_node.text)

PAGE 12Should You 
Learn Math to 
Get a Job in AI? CHAPTER 3
LEARNING

PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? It’s always nice to know more math! But there’s so much to 
learn that, realistically, it’s necessary to prioritize. Here’s how you might go about strengthening 
your math background.
To figure out what’s important to know, I find it useful to ask what you need to know to make 
the decisions required for the work you want to do. At DeepLearning.AI, we frequently ask, 
“What does someone need to know to accomplish their goals?” The goal might be building a 
machine learning model, architecting a system, or passing a job interview.
Understanding the math behind algorithms you use is often helpful, since it enables you to 
debug them. But the depth of knowledge that’s useful changes over time. As machine learning 
techniques mature and become more reliable and turnkey, they require less debugging, and a 
shallower understand

### Building the index

In [30]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [38]:
from llama_index import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

In [39]:
# NOTE: all nodes are stores in the docstore, but only leaf_nodes are included in the vectorstore index.
from llama_index import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [40]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
        leaf_nodes,
        storage_context=storage_context,
        service_context=auto_merging_context,
    )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context,
    )

### Defining the retriever and running the query engine

In [41]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(similarity_top_k=12)

retriever = AutoMergingRetriever(
    automerging_retriever, automerging_index.storage_context, verbose=True
)

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [42]:
auto_merging_response = auto_merging_engine.query(
    "What is the importance of networking in AI?"
)

In [43]:
from llama_index.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Networking is important in AI because it allows individuals to build a strong professional network and community. This network can provide valuable information, help with career advancement, and offer support and advice when needed. By connecting with others in the AI community, individuals can also increase their visibility and recognition for their expertise. Additionally, networking can lead to referrals for potential job opportunities. Building a community and fostering relationships within the AI field can be more beneficial than simply focusing on personal networking efforts.

## Putting it all Together

In [44]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [46]:
from llama_index.llms import OpenAI

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index",
)

In [47]:
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

## TruLens Evaluation

In [ ]:
from trulens_eval import Tru

Tru().reset_database()

### Two layers

In [ ]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048, 512],
)

In [ ]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [ ]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(auto_merging_engine_0, app_id="app_0")

In [ ]:
eval_questions = []
with open("generated_questions.text", "r") as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [ ]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [ ]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

In [ ]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

In [ ]:
Tru().run_dashboard()

### Three layers

In [ ]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048, 512, 128],
)

In [ ]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [ ]:
tru_recorder = get_prebuilt_trulens_recorder(auto_merging_engine_1, app_id="app_1")

In [ ]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

In [ ]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

In [ ]:
Tru().run_dashboard()